### 1. 라이브러리 임포트

In [95]:
import joblib
import numpy as np
import pandas as pd
import os

from sklearn.svm import SVC

### 2. 데이터 로드

In [ ]:
# df = pd.read_csv("train_data_200_user2_center_grab.csv")

file_names = [
    "0624-user1-device1-table1",
    "0723-user2-device1-table1",
    "0626-user3-device1-table1",
    "0625-user7-device1-table1"
]

df = pd.DataFrame()

# 각 CSV 파일을 순회하며 데이터프레임으로 불러오고 하나로 합치기
for file in file_names:
    temp_df = pd.read_csv("data/" + file + ".csv")
    df = pd.concat([df, temp_df], ignore_index=True)

### 3. (추가 기능) 원하는 라벨만 필터링
**이곳에서 `desired_labels` 리스트를 수정하여 원하는 동작만 학습시킬 수 있습니다.**

In [97]:
# 사용 가능한 전체 라벨 확인
print(f"Available labels: {df['class'].unique()}")

# 학습에 사용할 라벨을 리스트로 지정 (예: ['clap', 'grab'])
desired_labels = ['pinchL', 'pinchR'] # <--- 이 부분을 수정하세요

# 지정한 라벨에 해당하는 데이터만 필터링
df = df[df['class'].isin(desired_labels)].copy()

print(f"\nTraining with labels: {df['class'].unique()}")
print(f"Filtered data shape: {df.shape}")

Available labels: ['clap' 'flickL' 'flickR' 'grabL' 'grabR' 'pinchL' 'pinchR' 'touchL'
 'touchR']

Training with labels: ['pinchL' 'pinchR']
Filtered data shape: (40, 2401)


### 4. 라벨과 피처 분리 (수정됨)
컬럼 이름을 명시하여 안정성을 높였습니다.

In [98]:
y = df['class'].values
X_raw = df.drop('class', axis=1).values.astype(float)

print("X_raw shape:", X_raw.shape)
print("y shape:", y.shape)

X_raw shape: (40, 2400)
y shape: (40,)


### 5. FFT 변환 함수 (수정됨)
`main.py`와 일치하도록 `rfft`를 사용합니다.

In [99]:
def extract_fft_features(raw_row, fs=400, N=400):
    features = []
    num_axes = 6 # 6축 (accel 3 + gyro 3)
    for i in range(num_axes):
        segment = raw_row[i*N:(i+1)*N]
        
        # main.py와 동일하게 rfft 사용
        fft_vals = np.fft.rfft(segment)
        fft_mag = np.abs(fft_vals)
        freqs = np.fft.rfftfreq(N, d=1/fs)
        
        mask = (freqs >= 1) & (freqs <= 200)
        fft_selected = fft_mag[mask]
        fft_logged = np.log1p(fft_selected)
        features.extend(fft_logged)
        
    return np.array(features)

### 6. 모든 샘플에 대해 피처 추출

In [100]:
X_feat = np.array([extract_fft_features(row) for row in X_raw])

print("Raw data shape:", X_raw.shape)
print("FFT feature shape:", X_feat.shape)

Raw data shape: (40, 2400)
FFT feature shape: (40, 1200)


### 7. SVM 학습 (수정됨)
**가장 중요한 수정: 원시 데이터(X_raw)가 아닌 FFT 변환된 피처(X_feat)로 학습합니다.**

In [101]:
svm_model = SVC(kernel="linear", decision_function_shape="ovo", probability=True)

# FFT 변환된 피처로 모델을 학습시킵니다.
svm_model.fit(X_feat, y)

print("SVM model training complete.")

SVM model training complete.


### 8. 모델 저장

In [102]:
# 파일 이름에 어떤 데이터로 학습했는지 명시하는 것이 좋습니다.
if len(file_names) <= 1:
    model_filename = f"svm_model_{file}.pkl"
else:
    model_filename = f"svm_model_file_num_{len(file_names)}.pkl"

joblib.dump(svm_model, "model/" + model_filename)

print(f"Model saved to {model_filename}")

Model saved to svm_model_0625-user7-device1-table1.pkl
